<a href="https://colab.research.google.com/github/magicalmug/MDCountySchoolStatus/blob/main/Eugor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
1import random
import time

class Player:
    def __init__(self, name):
        self.name = name
        self.hp = 100
        self.max_hp = 100
        self.attack = 10
        self.gold = 0
        self.xp = 0
        self.level = 1
        self.level_cap = 30
        self.seen_monsters = {}

    @property
    def xp_to_next_level(self):
        return int(55 * (self.level ** 1.65))

    @property
    def inn_cost(self):
        return self.level * 5

    @property
    def weapon_upgrade_cost(self):
        return self.attack * 10

    @property
    def armor_upgrade_cost(self):
        return self.max_hp * 2

    def log_monster(self, monster):
        if monster['name'] not in self.seen_monsters:
            self.seen_monsters[monster['name']] = monster

    def level_up(self):
        while self.xp >= self.xp_to_next_level and self.level < self.level_cap:
            self.xp -= self.xp_to_next_level
            self.level += 1
            self.max_hp += 35
            self.hp = self.max_hp
            self.attack += 7
            print(f"\n✨ LEVEL UP! You reached Level {self.level}!")
            time.sleep(0.4)

    def show_status(self):
        print("\n" + "="*50)
        lvl_display = f"LVL: {self.level}" if self.level < 30 else "LVL: 30 (MAX)"
        print(f" HERO: {self.name.upper().ljust(15)} | {lvl_display}")
        print(f" ❤️  HP: {self.hp}/{self.max_hp} | ⚔️  ATK: {self.attack} | 💰 GOLD: {self.gold}")
        if self.level < 30:
            bar_size = 20
            progress = int((self.xp / self.xp_to_next_level) * bar_size)
            print(f" XP: [{ '#' * progress }{ '-' * (bar_size - progress) }] ({self.xp}/{self.xp_to_next_level})")
        print("="*50)

def combat(player, zone_name, monsters):
    m = random.choice(monsters).copy()
    player.log_monster(m)
    print(f"\n--- {zone_name}: {m['name']} appeared! ---")
    print(f"'{m['desc']}'")

    while m['hp'] > 0 and player.hp > 0:
        p_dmg = random.randint(player.attack - 3, player.attack + 3)
        if random.random() < 0.12: p_dmg *= 2; print("💥 CRITICAL!")
        m['hp'] -= p_dmg
        print(f"⚔️ You deal {p_dmg} dmg. ({m['name']}: {max(0, m['hp'])} HP)")

        if m['hp'] <= 0: break

        m_dmg = random.randint(m['atk'] - 4, m['atk'] + 4)
        if random.random() < 0.20:
            m_dmg = int(m_dmg * m.get('spec_mult', 1.5))
            print(f"⚠️  {m['name']} used {m['ability']}!")

        player.hp -= m_dmg
        print(f"🩸 {m['name']} deals {m_dmg} dmg. (You: {max(0, player.hp)} HP)")
        time.sleep(0.1)

    if player.hp > 0:
        penalty = 1 + (player.level / 12)
        final_gold = max(1, int(m['gold'] / penalty))
        print(f"\n✅ Victory! +{m['xp']} XP | +{final_gold} Gold")
        if player.level < 30: player.xp += m['xp']
        player.gold += final_gold
        player.level_up()
    else:
        print(f"\n💀 {player.name} died in {zone_name}. Final Level: {player.level}")
        exit()

# --- TOWN LOGIC ---
def beastmaster(p):
    print(f"\n--- THE BEASTMASTER'S JOURNAL ---")
    if not p.seen_monsters:
        print("Empty. Go find some trouble!")
    else:
        for name, data in p.seen_monsters.items():
            print(f"[{name.upper()}] - {data['desc']}")
            print(f" - HP: {data['hp']} | ATK: {data['atk']} | Skill: {data['ability']}")
            print("-" * 30)
    input("\n[Press Enter]")

def blacksmith(p):
    while True:
        print(f"\n--- EUGOR SMITHY (Gold: {p.gold}) ---")
        print(f"1. Sharpen Blade (+8 ATK)  - {p.weapon_upgrade_cost}g")
        print(f"2. Forge Plates  (+60 HP)  - {p.armor_upgrade_cost}g")
        print(f"3. Sell Scraps")
        print(f"4. Leave")
        c = input("> ")
        if c == "1" and p.gold >= p.weapon_upgrade_cost:
            p.gold -= p.weapon_upgrade_cost
            p.attack += 8
            print("⚔️ Attack Up!")
        elif c == "2" and p.gold >= p.armor_upgrade_cost:
            p.gold -= p.armor_upgrade_cost
            p.max_hp += 60
            p.hp += 60
            print("🛡️ HP Up!")
        elif c == "3":
            scrap = random.randint(p.level * 2, p.level * 10)
            p.gold += scrap
            print(f"💰 Sold bits for {scrap}g!")
        elif c == "4": break

def visit_town(p):
    while True:
        print(f"\n--- TOWN OF EUGOR ---")
        print(f"1. Inn ({p.inn_cost}g) | 2. Blacksmith | 3. Beastmaster | 4. Leave")
        c = input("> ")
        if c == "1" and p.gold >= p.inn_cost:
            p.gold -= p.inn_cost
            p.hp = p.max_hp
            print("🛌 Restored!")
        elif c == "2": blacksmith(p)
        elif c == "3": beastmaster(p)
        elif c == "4": break

# --- THE FIVE ZONES (Expanded) ---
zones = {
    "1": ("The Meadows", "Lv 1-6", [
        {"name": "Rabid Rabbit", "hp": 30, "atk": 6, "xp": 35, "gold": 15, "desc": "A fluffy ball of pure rage.", "ability": "Nibble", "spec_mult": 1.4},
        {"name": "Green Slime", "hp": 40, "atk": 5, "xp": 40, "gold": 20, "desc": "Wobbles with malicious intent.", "ability": "Dissolve", "spec_mult": 1.3},
        {"name": "Drunk Bandit", "hp": 55, "atk": 8, "xp": 50, "gold": 45, "desc": "He thinks you're a giant gold coin.", "ability": "Bottle Smash", "spec_mult": 1.5}
    ]),
    "2": ("Abandoned City", "Lv 7-12", [
        {"name": "Rusty Automaton", "hp": 160, "atk": 25, "xp": 180, "gold": 110, "desc": "Clockwork murder machines.", "ability": "Gear Grind", "spec_mult": 1.4},
        {"name": "Street Urchin", "hp": 120, "atk": 20, "xp": 150, "gold": 90, "desc": "Fast hands and a rusty shiv.", "ability": "Pickpocket", "spec_mult": 1.8},
        {"name": "Stone Gargoyle", "hp": 220, "atk": 30, "xp": 240, "gold": 150, "desc": "A statue that forgot how to stay still.", "ability": "Crushing Dive", "spec_mult": 1.5}
    ]),
    "3": ("Dangerous Desert", "Lv 13-18", [
        {"name": "Sand Worm", "hp": 450, "atk": 55, "xp": 500, "gold": 400, "desc": "Better stay on the rocks.", "ability": "Tremor", "spec_mult": 1.7},
        {"name": "Cactus Man", "hp": 380, "atk": 45, "xp": 450, "gold": 300, "desc": "A very prickly encounter.", "ability": "Needle Spray", "spec_mult": 1.6},
        {"name": "Mirage Wraith", "hp": 550, "atk": 65, "xp": 700, "gold": 500, "desc": "Heat made manifest.", "ability": "Solar Flare", "spec_mult": 2.0}
    ]),
    "4": ("The Dungeon", "Lv 19-24", [
        {"name": "Dark Knight", "hp": 1200, "atk": 110, "xp": 1500, "gold": 1200, "desc": "Armor filled with shadow.", "ability": "Cleave", "spec_mult": 1.5},
        {"name": "Beholder", "hp": 900, "atk": 140, "xp": 1800, "gold": 1500, "desc": "Beauty is in the eye of the... oh no.", "ability": "Disintegration", "spec_mult": 2.2},
        {"name": "Mimic Chest", "hp": 1500, "atk": 100, "xp": 2000, "gold": 3000, "desc": "That's not loot.", "ability": "Crunch", "spec_mult": 1.9}
    ]),
    "5": ("Fires of the Abyss", "Lv 25-30", [
        {"name": "Void Overlord", "hp": 4500, "atk": 380, "xp": 8000, "gold": 10000, "desc": "The king of nothing.", "ability": "Reality Warp", "spec_mult": 3.0},
        {"name": "Hellfire Hydra", "hp": 3500, "atk": 300, "xp": 6500, "gold": 7000, "desc": "Five heads, zero mercy.", "ability": "Inferno", "spec_mult": 2.2},
        {"name": "Fallen Angel", "hp": 5000, "atk": 420, "xp": 10000, "gold": 15000, "desc": "Divine power, corrupted.", "ability": "Judgement", "spec_mult": 2.5}
    ])
}

# --- MAIN LOOP ---
hero = Player(input("Name your hero: "))
while True:
    hero.show_status()
    print("DESTINATIONS:")
    for k, v in zones.items():
        print(f" {k}. {v[0].ljust(20)} (Range: {v[1]})")
    print(" 6. Travel to Eugor (Town)")
    print(" 7. Quit")

    choice = input("\nSelect: ")
    if choice in zones:
        combat(hero, zones[choice][0], zones[choice][2])
    elif choice == "6": visit_town(hero)
    elif choice == "7": break


 HERO: B               | LVL: 1
 ❤️  HP: 100/100 | ⚔️  ATK: 10 | 💰 GOLD: 0
 XP: [--------------------] (0/55)
DESTINATIONS:
 1. The Meadows          (Range: Lv 1-6)
 2. Abandoned City       (Range: Lv 7-12)
 3. Dangerous Desert     (Range: Lv 13-18)
 4. The Dungeon          (Range: Lv 19-24)
 5. Fires of the Abyss   (Range: Lv 25-30)
 6. Travel to Eugor (Town)
 7. Quit

--- TOWN OF EUGOR ---
1. Inn (5g) | 2. Blacksmith | 3. Beastmaster | 4. Leave

--- THE BEASTMASTER'S JOURNAL ---
Empty. Go find some trouble!

--- TOWN OF EUGOR ---
1. Inn (5g) | 2. Blacksmith | 3. Beastmaster | 4. Leave

 HERO: B               | LVL: 1
 ❤️  HP: 100/100 | ⚔️  ATK: 10 | 💰 GOLD: 0
 XP: [--------------------] (0/55)
DESTINATIONS:
 1. The Meadows          (Range: Lv 1-6)
 2. Abandoned City       (Range: Lv 7-12)
 3. Dangerous Desert     (Range: Lv 13-18)
 4. The Dungeon          (Range: Lv 19-24)
 5. Fires of the Abyss   (Range: Lv 25-30)
 6. Travel to Eugor (Town)
 7. Quit

--- The Meadows: Drunk Bandit a